# Welcome to ProgPy's Serialization Tutorial!

In this tutorial, we will learn how to serialize and deserialize Python objects using the `pickle` and `JSON` methods.

Model serialization has a lot of purposes, like saving a specific model to a file to be loaded later or sending a model to another machine over a network connection.

### Importing Modules

In [ ]:
import matplotlib.pyplot as plt
import pickle
from progpy.models import BatteryElectroChemEOD as Battery

We'll need to create a model object. For this example, we'll use the [BatteryElectroChemEOD](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#battery-model) model!

Furthermore, we'll be setting our `process_noise` to 0 to illustrate the match between original and serialized versions!

In [ ]:
batt = Battery()
batt.parameters['process_noise'] = 0 

##### __METHOD 1: Serialize with Pickle__

First, let's demonstrate how to serialize a model using the `pickle` package.

In [ ]:
pickle.dump(batt, open('model_save_pkl.pkl','wb'))
load_pkl = pickle.load(open('model_save_pkl.pkl','rb'))

##### __METHOD 2: Serialize with JSON__

Next, let's demonstrate how to serialize a model using the `to_json` method.

In [ ]:
save_json = batt.to_json()

We can call the model directly with the serialized result using the `from_json` method.

In [ ]:
serial_1 = Battery.from_json(save_json)

Serialized result can also be saved to a text file and uploaded later using the following code:

In [ ]:
txtFile = open("model_save_json.txt", "w")
txtFile.write(save_json)
txtFile.close()

with open('model_save_json.txt') as infile: 
    load_json = infile.read()

serial_2 = Battery.from_json(load_json)

### Comparing the simulations of the serialized models

We can use ProgPy's [Simulation](https://nasa.github.io/progpy/prog_models_guide.html#simulation) methods to simulate the model to threshold and compare the results of the serializations.

First, we'll need to define our [Future Loading Profile](https://nasa.github.io/progpy/prog_models_guide.html#future-loading).

In [ ]:
def future_loading(t, x=None):
    if (t < 600):
        i = 3
    elif (t < 1000):
        i = 2
    elif (t < 1500):
        i = 1.5
    else:
        i = 4
    return batt.InputContainer({'i': i})

Now, let's simulate to threshold using the `simulate_to_threshold` method.

In [ ]:
results_orig = batt.simulate_to_threshold(future_loading, save_freq = 1)
results_serial_1 = serial_1.simulate_to_threshold(future_loading, save_freq = 1)
results_serial_2 = serial_2.simulate_to_threshold(future_loading, save_freq = 1)
results_serial_3 = load_pkl.simulate_to_threshold(future_loading, save_freq = 1)

Now, let's plot the results for comparison.

In [ ]:
voltage_orig = [results_orig.outputs[iter]['v'] for iter in range(len(results_orig.times))]
voltage_serial_1 = [results_serial_1.outputs[iter]['v'] for iter in range(len(results_serial_1.times))]
voltage_serial_2 = [results_serial_2.outputs[iter]['v'] for iter in range(len(results_serial_2.times))]
voltage_serial_3 = [results_serial_3.outputs[iter]['v'] for iter in range(len(results_serial_3.times))]

plt.plot(results_orig.times,voltage_orig,'-b',label='Original surrogate') 
plt.plot(results_serial_1.times,voltage_serial_1,'--r',label='First JSON serialized surrogate') 
plt.plot(results_serial_2.times,voltage_serial_2,'-.g',label='Second JSON serialized surrogate') 
plt.plot(results_serial_3.times, voltage_serial_3, '--y', label='Pickled serialized surrogate')
plt.legend()
plt.xlabel('Time (sec)')
plt.ylabel('Voltage (volts)')

Since there is some overlap on the graph, we can also determine that the results are the same by comparing the output arrays directly.

In [ ]:
import numpy as np

# Check if the arrays are the same
are_arrays_same = np.array_equal(voltage_orig, voltage_serial_1) and \
                  np.array_equal(voltage_orig, voltage_serial_2) and \
                  np.array_equal(voltage_orig, voltage_serial_3)

print(f"The simulated results from the original and serialized models are {'identical. This means that our serialization works!' if are_arrays_same else 'not identical. This means that our serialization does not work.'}!")


## Conclusion

In this notebook, we explored how serialization can be used in ProgPy. We learned how to utilize the `pickle` module to serialize and deserialize ProgPy models. We also explored alternative serialization formats like JSON.

Overall, understanding how to serialize and deserialize models can be a powerful tool for Prognostics developers. It allows you to save your models to disk and load them back into memory at a later time. 

For more information on ProgPy, please refer to the ProgPy [Documentation](https://nasa.github.io/progpy/).